<a href="https://colab.research.google.com/github/safin92/dz6_xor_voprosi/blob/main/XOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задача XOR

# Задача: Реализация нейронной сети для решения проблемы XOR

## Введение

XOR (исключающее ИЛИ) - это логическая операция, которая возвращает истину только если входные значения различны. Таблица истинности для XOR:

| A | B | A XOR B |
|:-:|:-:|:-------:|
| 0 | 0 |    0    |
| 0 | 1 |    1    |
| 1 | 0 |    1    |
| 1 | 1 |    0    |

Эта задача не может быть решена с помощью линейной модели, поэтому она часто используется для демонстрации возможностей нейронных сетей.

## Задание

Ваша задача - реализовать двухслойную нейронную сеть для решения проблемы XOR.

### Архитектура сети:

- Входной слой: 2 нейрона (для A и B)
- Скрытый слой: 4 нейрона
- Выходной слой: 1 нейрон

![Архитектура нейронной сети для XOR](Tutorial-And.jpg)

## Инструкции:

1. Изучите предоставленный код класса `NeuralNetwork`.
2. Заполните все места, отмеченные `TODO`, используя предоставленные подсказки.
3. Используйте функции NumPy (`np.dot`, `np.random.uniform`, `np.sum`) для выполнения необходимых вычислений.
4. Реализуйте прямое и обратное распространение, а также обновление весов.
5. Обучите сеть на предоставленных данных XOR.
6. Протестируйте сеть и выведите результаты.

In [1]:
import numpy as np

In [16]:
# Класс нейронной сети с одним скрытым слоем
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Инициализация параметров сети
        self.input_size = input_size      # Размер входного слоя (количество нейронов (признаков))
        self.hidden_size = hidden_size    # Количество нейронов в скрытом слое
        self.output_size = output_size    # Размер выходного слоя

        # Шаг 1: Инициализация весов и смещений
        # Весовые коэффициенты и смещения инициализируются случайным образом для разрыва симметрии в начальных значениях
        self.hidden_weights = np.random.uniform(size=(input_size, hidden_size)) # Веса между входным и скрытым слоем
        self.hidden_bias = np.random.uniform(size=(1, hidden_size))              # Смещения для скрытого слоя
        self.output_weights = np.random.uniform(size=(hidden_size, output_size)) # Веса между скрытым и выходным слоем
        self.output_bias = np.random.uniform(size=(1, output_size))              # Смещения для выходного слоя

    # Шаг 2: Функция активации - Сигмоида
    # Используется для нелинейности в нейронной сети
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Шаг 3: Производная сигмоиды
    # Используется для вычисления градиентов при обратном распространении ошибки
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    # Шаг 4: Прямое распространение (Forward Propagation)
    # Вычисление выходных значений сети на основе входных данных
    def forward(self, X):
        # Входной слой -> Скрытый слой
        # Применяем веса и смещения, затем функцию активации
        self.hidden_layer = self.sigmoid(np.dot(X, self.hidden_weights) + self.hidden_bias)

        # Скрытый слой -> Выходной слой
        # Применяем веса и смещения, затем функцию активации
        self.output_layer = self.sigmoid(np.dot(self.hidden_layer, self.output_weights) + self.output_bias)

        # Возвращаем выходное значение
        return self.output_layer

    # Шаг 5: Обратное распространение ошибки (Backward Propagation)
    # Обновление весов и смещений на основе ошибки предсказания
    def backward(self, X, y, output):
        # Шаг 5.1: Вычисление ошибки выходного слоя
        # Разница между фактическим и предсказанным значением
        error = y - output

        # Шаг 5.2: Вычисление градиента выходного слоя
        # Умножаем ошибку на производную функции активации
        d_output = error * self.sigmoid_derivative(output)

        # Шаг 5.3: Распространение ошибки на скрытый слой
        # Умножаем градиенты выходного слоя на веса выходного слоя
        error_hidden_layer = np.dot(d_output, self.output_weights.T)

        # Шаг 5.4: Вычисление градиента скрытого слоя
        # Умножаем ошибку скрытого слоя на производную функции активации
        d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(self.hidden_layer)

        # Шаг 5.5: Обновление весов и смещений
        # Используем градиентный спуск для корректировки весов и смещений
        # Выходной слой
        self.output_weights += np.dot(self.hidden_layer.T, d_output) * self.learning_rate
        self.output_bias += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate

        # Скрытый слой
        self.hidden_weights += np.dot(X.T, d_hidden_layer) * self.learning_rate
        self.hidden_bias += np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate

    # Шаг 6: Обучение нейронной сети
    # Повторение этапов прямого и обратного распространения для каждой эпохи
    def train(self, X, y, epochs, learning_rate):
        self.learning_rate = learning_rate  # Устанавливаем скорость обучения
        for _ in range(epochs):
            output = self.forward(X)        # Прямое распространение
            self.backward(X, y, output)     # Обратное распространение

    # Шаг 7: Предсказание
    # Вычисление выхода для новых данных без обучения
    def predict(self, X):
        return self.forward(X)


In [17]:
# Наш "датасет" для обучения
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])


In [49]:
# Инициализация нейронной сети
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)
#Цикл обучения
nn.train(X, y, epochs = 1000, learning_rate=10)

In [50]:
predictions = nn.predict(X)
print("Выходные данные после обучения:")
print(predictions)

# Подсказка: Используйте np.mean для вычисления среднего значения
# np.mean вычисляет среднее арифметическое элементов массива
accuracy = np.mean(np.round(predictions) == y)
print(f"\nТочность: {accuracy * 100}%")

Выходные данные после обучения:
[[0.00501534]
 [0.98800521]
 [0.98816653]
 [0.01792892]]

Точность: 100.0%


**1. Как называется алгоритм нейронный сетей.**

Градиентный спуск (или обртаное распространение ошибки). Элементы алгоритма включают в себя:
  
  1) Прямое распространение: данные проходят через сеть, от входного слоя к выходному - каждый нейрон взвешивает сумму входов, добавлет смещение и пропускает результат через функцию активации - на выходе сеть дает предсказание.
  
  2) Вычисление ошибки: сравниваем предсказание сети с правильным ответом и вычисляем ошибку (MSE, крос-энтропия).
  
  3) Обратное распространение ошибки: ошибка распространяется назад по сети, начиная с выходного слоя - вычисляются градиенты (производные функции по всем весам).
  
  4) Обновление весов: градиентный спуск корректирует веса так, чтобы уменьшить ошибку - вес обновляется по формуле (новый вес = старый вес - скорость обучения*градиент ошибки по весу.


**2. По какому методу градиентного спуска реализован алгоритм?**

Методы градиентного спуска:
  
  1) Полный (Batch) градиентный спуск (Batch Gradient Descent, BGD)

Обновляет веса раз в эпоху (полный проход посети), используя всю выборку данных.
Точный, но медленный и требует много памяти.
Хорош для гладких функций (например, в линейной регрессии).

  2) Стохастический градиентный спуск (Stochastic Gradient Descent, SGD)

Обновляет веса после каждого примера (пример - отдельный вход, один объект данных , который подается на вход нейросети).
Быстрее, но из-за случайности обновлений может колебаться и не всегда сходится к оптимальному минимуму.
Хорош для онлайн-обучения.

  3) Мини-батч градиентный спуск (Mini-Batch Gradient Descent, MBGD)

Компромисс между BGD и SGD. Разбивает выборку на маленькие группы (батчи) и обновляет веса после обработки каждого батча.
Быстрее, чем BGD, и менее шумный, чем SGD.

  4) Adagrad (Adaptive Gradient Descent)

Разные параметры обучаются с разной скоростью.
Хорошо работает для разреженных данных.
Проблема: скорость обучения со временем становится слишком маленькой.

  5) RMSprop (Root Mean Square Propagation)

Улучшает Adagrad: добавляет затухающее среднее квадратов градиентов.
Хорошо работает в глубоких сетях.

  6) Adam (Adaptive Moment Estimation) – самый популярный

Комбинирует Momentum и RMSprop.
Самый распространенный метод в нейросетях.
Автоматически регулирует скорость обучения для каждого параметра.

Какой метод выбрать?
Если данных мало → BGD (полный градиентный спуск).
Если данных много → Mini-Batch или Adam.
Для онлайн-обучения → SGD.
Для глубоких сетей → Adam или RMSprop.

В коде реализован полный (Batch) градиентный спуск, так как веса обновляются после обработки всего набора данных.

**3. Как подбирать Learning_rate**

Скорость обучения - коэффициент, который определяет, насколько сильно обновляются веса нейросети на каждом шаге обучения.

Выбор learning_rate критически важен:

Слишком маленький α → обучение идет очень медленно, можно застрять в локальном минимуме.
Слишком большой α → веса обновляются слишком резко, может начаться "скачкообразное" обучение или сеть вообще не сойдется.

Методы:

1️⃣ Метод проб и ошибок (эмпирический поиск)
Простейший способ — начать с небольшого значения (0.01 или 0.001) и наблюдать за изменением ошибки.

Если ошибка уменьшается медленно → попробуй увеличить α.
Если ошибка скачет или растет → уменьшай α.
Пример:

nn.train(X, y, epochs=1000, learning_rate=0.01)

Начни с 0.01. Если градиент "скачет" → уменьши α (0.001).
Если сходится медленно → попробуй 0.05 или 0.1.

2️⃣ Метод логарифмического поиска (поиск по степеням 10)
Попробуй разные значения α в диапазоне от 1e-5 до 1e-1 (от 0.00001 до 0.1):

python
Копировать
Редактировать
for lr in [0.00001, 0.0001, 0.001, 0.01, 0.1]:
    print(f"Тестируем learning_rate = {lr}")
    nn.train(X, y, epochs=1000, learning_rate=lr)
💡 Если α = 0.1 работает хорошо, попробуй 0.05 или 0.2 для уточнения.

3️⃣ Адаптивные методы (Adam, RMSprop, Adagrad)
Можно использовать автоматическую подстройку learning_rate:

Adam — один из лучших оптимизаторов (сочетает Momentum + RMSprop).
RMSprop — хорошо работает на сложных задачах.
Adagrad — подходит, если данные сильно различаются по масштабам.
Пример с Adam в Keras:

python
Копировать
Редактировать
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.01)


4️⃣ Эксперимент с "learning rate decay" (уменьшение скорости со временем)
Иногда обучение сначала требует большого α, но потом его лучше уменьшить. Это называется "learning rate decay" (затухание скорости обучения).
Пример:

python
Копировать
Редактировать
initial_lr = 0.1
decay_rate = 0.01
lr = initial_lr / (1 + decay_rate * epoch)  # Уменьшаем lr по мере роста эпох
💡 Чем дальше обучение, тем меньше шаги обновления весов, что помогает точнее настроить сеть.

5️⃣ Визуальный анализ (график ошибки)
Можно строить график ошибки (loss function) в зависимости от эпох.
Пример:

python
Копировать
Редактировать
import matplotlib.pyplot as plt

losses = []
for epoch in range(1000):
    output = nn.forward(X)
    loss = np.mean((y - output) ** 2)  # Среднеквадратичная ошибка (MSE)
    losses.append(loss)
    nn.backward(X, y, output)

plt.plot(losses)
plt.xlabel("Эпохи")
plt.ylabel("Ошибка (loss)")
plt.show()
Если кривая падает медленно → увеличь α.
Если скачет или растет → уменьши α.
🔥 Вывод:
Начинай с α = 0.01 или 0.001, тестируй.
Используй логарифмический поиск (0.00001 → 0.0001 → 0.001 → 0.01 → 0.1).
Для сложных моделей попробуй Adam или RMSprop вместо стандартного градиентного спуска.
Можно уменьшать α со временем (learning rate decay).
Строй графики ошибки, чтобы видеть, как α влияет на обучение.
Лучший learning_rate зависит от данных! 🚀

4) np.random.uniform

np.random — это модуль в NumPy, который содержит множество функций для генерации случайных чисел.

np.random.uniform — это конкретная функция из этого модуля, которая генерирует числа из равномерного распределения.


5) веса задать нулями и смешения тоже

Если в вашей задаче начальные веса задать нулями, нейросеть не сможет обучаться правильно.

Почему?
В ходе обратного распространения ошибки (backpropagation) все нейроны в одном слое будут обновляться одинаково, то есть они будут полностью симметричными. В результате:

Все нейроны в скрытом слое останутся идентичными — они будут получать одинаковые градиенты и обновляться одинаково.
Сеть не научится выделять разные признаки — разные нейроны не смогут научиться различным шаблонам в данных.
Итог
Модель просто "застрянет" и не сможет эффективно обучаться. Именно поэтому веса инициализируют случайными значениями, чтобы нарушить симметрию и позволить каждому нейрону обучаться по-разному.

6) +=, почему нету -=, шаг 5.5

В вашей задаче оператор += используется для обновления весов в процессе градиентного спуска:

python
Копировать
Редактировать
self.output_weights += np.dot(self.hidden_layer.T, d_output) * self.learning_rate
self.output_bias += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate

self.hidden_weights += np.dot(X.T, d_hidden_layer) * self.learning_rate
self.hidden_bias += np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate
Почему +=, а не -=?
На самом деле, в градиентном спуске классическая формула обновления весов выглядит так:

𝑊
=
𝑊
−
𝜂
⋅
∂
𝐿
∂
𝑊
W=W−η⋅
∂W
∂L
​

где:

𝑊
W — веса,
𝜂
η (learning rate) — скорость обучения,
∂
𝐿
∂
𝑊
∂W
∂L
​
  — градиент функции ошибки (L — loss).
Таким образом, обычно мы вычитаем градиент (-=), чтобы двигаться в сторону убывания ошибки.

Возможные причины использования +=
Ошибка в коде — возможно, += написано случайно, и нужно поменять на -=.
Обратный градиентный спуск (если формула градиента была рассчитана со знаком минус заранее, то += эквивалентно -=).
Использование специальной модификации градиентного спуска, но в этом коде это вряд ли.
Итог
Лучше заменить += на -= в строках обновления весов, чтобы соответствовать стандартному алгоритму градиентного спуска:

7) Регуляризация (это не вопрос), так для себя
8) попробовать софтмакс вместо сигмоиды, обучить софтмаксом
